## 군집합 분석 -> k-평균 알고리짐(군집합)분석
### 샘플 데이터 : 온라인 판매 데이터(영국 특정 쇼핑몰의 샘플 데이터 기반)
### UCI 레포지토리 다운 받을 예정. 
### 분석 모델 구축 : 사이킷런의 내부에 있는 패키지를 이용하고 있는 중. 
### 구글, 메타, 파이썬의 내장 패키지등, 오픈 소스를 이용하고 있다. 

### 용어정리 
### 비지도 학습: 쉽게 말해서, 답이 없는 학습을 시켜서, 학습된 모델을 이용해서, 예측값 , 실제값 비교하는 
### 간단한 프로세스 
### x(훈련데이터)-> 머신러닝 프로그램 -> 학습 모델 -> 클러스터(군, 집합, 분류..)
###  새로운 데이터 -> 학습 모델 -> 예측 레이블(타겟, 결과등)

### 예) 쇼핑몰 , 소비 패턴을 파악하는 비지도학습 모델링. 
### 예2) 겨울에 1월 시즌에는 어는 물품이 많이 구매가 되었는지, 연령층 별로, 구매 패턴-> 분류를 
### 예3) 30 ~ 40대, 책 구매를 많이했고, 식료품에 다이어트 식품들이 구매가 많이 되었다. -> 광고,  

### k-평균 알고리즘 
### k개의 클러스터를 구성한다. k개의 중심점을 임의의 위치로 잡고 중심점을 기준으로 가까이 있는 데이터를
### 확인한 뒤, 그들과의 거리의 평균 지점을 중심점으로 이동한다. -> 나중에, 삼각형 모양으로 중심점 표기.
### 임의의 클러스터의 k개의 최적 갯수를 찾는 부분이 문제점
### 가장 좋은 k를 찾기 위한 방법으로, 엘보 방법, 실루엣 방법을 사용할 예정. 

### 엘보 방법. 
### 클러스터의 중심점과 클러스터 내의 데이터 거리 차이의 제곱값 합을 -> 왜곡이라고 부름 
### 선이 급격히 꺽이는 지점 있음. 그 지점 엘보(팔꿈치), 값을 최적의 k 값으로 사용함 -> 그래프로 꺽이는 지점, 보여드림. 

### 실루엣 분석
### 클러스터의 갯수를 얼마로 정할지를 지정해주는 분석 방법.
### 클러스터 내에 있는 데이터가 얼마나 조밀하게 모여 있는지를 측정하는 도구. 
### 데이터 i가 해당 클러스터(군, 집합)내의 데이터와 얼마나 가까운지를 나타내는 응집력 a(i)
### 가장 가까운 다른 클러스터 내의 데이터와 얼마나 떨어져 있는지 (분리도) b(i)
### 실루엣 계수 s(i)-> = (b(i) - a(i)) / max (a(i), b(i)) , 
### 실루엣 계수의 결괏값은 , -1 < s(i) <1, 결론, -> 1에 가까울수록 좋은 군집화. 


### 사이킷런에서, 함수로 다 제공해주니, 이용하기.



### 샘플 데이터 받기
### UCI 레포지토리에서 -> online retail 


In [1]:
# 엑셀 파일 읽기 도구 설치. 
!pip install openpyxl

In [2]:
import pandas as pd 
import math

retail_df = pd.read_excel("./Online_Retail.xlsx")
retail_df.head()